# Workbench for ResumeAssistant

In [1]:
from pdf2image import convert_from_path
# ImageWorker takes in the path to jpg resume image and returns the contents of the resume in a json format. 
from resumeassistant.operator.worker import ImageWorker
from resumeassistant.
from openai import OpenAI
import json

# PDF to jpg image converter function
This function takes the input path for pdf-resume and output path for resume image. The output images are stored in 'jpg' format. 

In [2]:
def get_img(pdf_path, img_path):
      # Method for converting pdf resumes to jpg images. 
      images = convert_from_path(pdf_path)
      # print(type(images))
      for i in range(len(images)):
        
            # Save pages as images in the pdf
          images[i].save(img_path + str(i) +'.jpg', 'JPEG')

This is the code to extract OpenAI key stored in `gpt_key.json` file. \
Format: \
    ```{ 
	"api_key": \<open_ai_key\>
    }```

In [3]:
with open('gpt_key.json', 'r') as file:
      data = json.load(file)
      api_key = data["api_key"]

## Resume Parsing
This is the code for converting the PDF resume to an image and parsing it using "GPT-4o". 

In [4]:
client = OpenAI(api_key=api_key)
get_img('Harish_Resume_PhD.pdf', './Harish_Resume')

sys_prompt = "You are a resume parser. Given the image of the resume, parse every section and provide the output in a JSON format."
user_prompt="Can you parse all sections of this resume?"

image_worker = ImageWorker(client=client, model_id="gpt-4o-mini", sys_prompt=sys_prompt, user_prompt=user_prompt)
parsed_resume = image_worker.get_output('Harish_Resume0.jpg')
print(parsed_resume)

{'name': 'Harish Sista', 'contact': {'phone': '(201) 310-5683', 'email': 'hsista@stevens.edu', 'address': '2642 28th St, Apt 4F, Astoria, NY-11102'}, 'summary': 'Ph.D. Candidate researching in ML and NLP with 7+ years of experience working with ML models, Big Data, and Cloud Computing platforms and 2+ years of experience working in a professional environment. Spearheaded an Evidence Extraction and Fact Verification model using Prompt Engineering and tested it with various LLMs. Invented a Black Box Optimization model for online hyperparameter fine-tuning, significantly enhancing model performance in real-time applications.', 'skills': {'programming_languages': ['Python', 'C++', 'Java', 'Objective C', 'Swift', 'SQL'], 'ai_tools': ['PyTorch', 'OpenAI', 'HuggingFace', 'Transformers', 'TensorFlow', 'SciKitLearn', 'NLTK', 'NumPy'], 'llms': ['GPT', 'LLAMA', 'MISTRAL', 'BLOOM', 'BERT-based models'], 'developer_apis': ['Twitter', 'Facebook', 'YouTube', 'Google', 'Amazon'], 'cloud_platforms': [

## Add CV and other information
The current_cv instance keeps a record of the most recent updated Cover Letter and additional information on work experience, projects, achievements, and certifications. 

### Input arguments:
`resume`=Parsed resume (Dict). \
`cv` = Cover letter text. \
`add_skills` = Additional skills not mentioned in the resume (Dict). \
`add_work_experience` = Additional work experience not mentioned in the resume (Dict). \
`add_education` = Additional education not mentioned in the resume (Dict). \
`add_projects` = Additional Projects not mentioned in the resume (Dict). \
`add_achievements` = Additional achievements not mentioned in the resume (Dict). \
`add_certifications` = Additional certifications not mentioned in the resume (Dict). 

#### additional_information format:
```{``` \
```"information": text,``` \
```"organization": text,``` \
```"date": "dd/mm/yyyy - dd/mm/yyyy" (or) "dd/mm/yyyy"``` \
```}```

The job_listing instance keeps a record of the requirements and description of the job posting. 

In [6]:
print(parsed_resume.keys())

dict_keys(['name', 'contact', 'summary', 'skills', 'education', 'research_experience', 'work_experience'])
